# LLM Iteration 15

**Aim**

To add a summary function to the StrokeWard class that outputs the mean waiting time for a bed and mean number of patients waiting for a bed.

**Prompt**

Update this code to also collect the mean number of patients waiting for a bed in the stroke ward during the course of the model. Also create a summary function inside the stroke ward class. This prints out the mean waiting time for a bed during the run and the mean number of patients waiting for a bed in a pandas dataframe.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**LLM Output**

To update the code to collect the mean number of patients waiting for a bed in the stroke ward and create a summary function inside the StrokeWard class to print out the mean waiting time and the mean number of patients waiting for a bed in a pandas DataFrame, you can make the following modifications:

In [1]:
import simpy
import numpy as np
import math
import pandas as pd

In [2]:
#### MODIFICATION - code from previous iter ####
class Scenario:
    def __init__(self):
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self):
        interarrival_distributions = {}
        for patient_type, seed in self.random_seeds.items():
            interarrival_distributions[patient_type] = \
                np.random.default_rng(seed)
        return interarrival_distributions
################################################

In [3]:
class StrokeWard:
    def __init__(self, env, num_beds, scenario):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.create_distributions()
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))
            print(f'Patient {patient.id} ({patient.patient_type}) admitted to '
                  f'the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')
            mu, sigma = calculate_mu_sigma(self.length_of_stay[patient.\
                                           patient_type]["mean"],
                                           self.length_of_stay[patient.\
                                           patient_type]["std_dev"])
            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))

            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################

            print(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())
            route_choice = np.random.choice(route_names, p=route_probabilities)
            print(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].\
                    exponential(scenario.\
                                default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)
        
        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }
        
        summary_df = pd.DataFrame(summary_data)
        
        print("Summary:")
        print(summary_df)

# Rest of the code remains unchanged

#### MODIFICATION - code from previous iter ####
class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma
################################################

In [4]:
env = simpy.Environment()
scenario = Scenario()
#### MODIFICATION - removed num_beds=10 due to error ####
# 'positional argument follows keyword argument'
############################################################
stroke_ward = StrokeWard(env, 10, scenario)

for patient_type in scenario.random_seeds.keys():
    env.process(stroke_ward.patient_arrival(patient_type))

# Set the run length to 2 months (default)
run_length_days = round(60 * 2)

print("Simulation started...")
env.run(until=run_length_days)
print("Simulation ended.")

stroke_ward.summary()

Simulation started...
Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.49
Patient 0 (Complex Neurological) routed to: Other
Patient 2 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 3 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 4 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.38
Patient 1 (Stroke) routed to: Other
Patient 3 (Stroke) discharged from the stroke ward at day 2.86
Patient 3 (Stroke) routed to: Other
Patient 5 (Stroke) admitted to the stroke ward at day 3.08. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 3.32. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 5.03. Wait time: 0 days
Patient 8 (Stroke) a

In this updated code snippet:
- The waiting_patients list is introduced to track the number of patients waiting for a bed in the stroke ward.
- The summary method is added to calculate and print out the mean wait time and the mean number of patients waiting for a bed in a pandas DataFrame.
- After running the simulation for the specified duration, the summary method is called to display the summary statistics.
- The rest of the code remains unchanged with patient arrivals and simulation execution as before.

**Checks**

In [5]:
class TestCase:
    def __init__(self, env, ward):
        self.env = env
        self.ward = ward
        for patient_type in self.ward.interarrival_distributions:
            env.process(self.ward.patient_arrival(patient_type))

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.run(until=RUN_LENGTH)
        self.ward.summary()

        if self.env.now > RUN_LENGTH:
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        self.env.run(until=RUN_LENGTH)
        self.ward.summary()

        for bed_use in ward.bed_in_use_store:
            if bed_use > NUM_BEDS:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        self.env.run(until=RUN_LENGTH)
        self.ward.summary()

        print('\nLength of stay:')
        for pat_type, los in zip(ward.patient_type_store, ward.los_store):
            print(f'({pat_type}) {los}')

In [6]:
# Checks run length
NUM_BEDS = 10
RUN_LENGTH = 10

scenario = Scenario()
env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS, scenario)

test = TestCase(env, ward)
test.test_run_length()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.49
Patient 0 (Complex Neurological) routed to: Other
Patient 2 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 3 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 4 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.38
Patient 1 (Stroke) routed to: Other
Patient 3 (Stroke) discharged from the stroke ward at day 2.86
Patient 3 (Stroke) routed to: ESD
Patient 5 (Stroke) admitted to the stroke ward at day 3.08. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 3.32. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 5.03. Wait time: 0 days
Patient 8 (Stroke) admitted to the stroke wa

In [7]:
# Checks that is only ever 1 patient if there is only 1 bed
NUM_BEDS = 1
RUN_LENGTH = 60

scenario = Scenario()
env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS, scenario)

test = TestCase(env, ward)
test.test_bed_hold()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.49
Patient 0 (Complex Neurological) routed to: Other
Patient 1 (Stroke) admitted to the stroke ward at day 0.49. Wait time: 0.27 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.65
Patient 1 (Stroke) routed to: Rehab Unit
Patient 2 (Stroke) admitted to the stroke ward at day 2.65. Wait time: 1.66 days
Patient 2 (Stroke) discharged from the stroke ward at day 4.01
Patient 2 (Stroke) routed to: Other
Patient 3 (Stroke) admitted to the stroke ward at day 4.01. Wait time: 2.51 days
Patient 3 (Stroke) discharged from the stroke ward at day 4.8
Patient 3 (Stroke) routed to: Other
Patient 4 (TIA) admitted to the stroke ward at day 4.8. Wait time: 2.78 days
Patient 4 (TIA) discharged from the stroke ward at day 9.05
Patient 4 (TIA) routed to: Other
Patient 5 (Stroke) admitted to the stroke ward at day 9.05. Wait ti

In [8]:
# Checks length of stay for patients
NUM_BEDS = 10
RUN_LENGTH = 10

scenario = Scenario()
env = simpy.Environment()
ward = StrokeWard(env, NUM_BEDS, scenario)

test = TestCase(env, ward)
test.test_length_of_stay()

Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: 0 days
Patient 1 (Stroke) admitted to the stroke ward at day 0.22. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.49
Patient 0 (Complex Neurological) routed to: Other
Patient 2 (Stroke) admitted to the stroke ward at day 0.99. Wait time: 0 days
Patient 3 (Stroke) admitted to the stroke ward at day 1.5. Wait time: 0 days
Patient 4 (TIA) admitted to the stroke ward at day 2.02. Wait time: 0 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.38
Patient 1 (Stroke) routed to: Other
Patient 3 (Stroke) discharged from the stroke ward at day 2.86
Patient 3 (Stroke) routed to: Other
Patient 5 (Stroke) admitted to the stroke ward at day 3.08. Wait time: 0 days
Patient 6 (Stroke) admitted to the stroke ward at day 3.32. Wait time: 0 days
Patient 7 (Other) admitted to the stroke ward at day 5.03. Wait time: 0 days
Patient 8 (Stroke) admitted to the stroke 

**Reflection**

The LLM has created a summary function inside the StrokeWard class as instructed. It is also capturing the number of patients waiting in the queue at point when a bed is yielded. The same results from the test case are being out from the previous iteration. However, the choice of destination is still random for patients. 